In [1]:
import numpy as np 
from scipy.optimize import minimize
from itertools import combinations
from tqdm import tqdm

Back to the origin the problem, we recast the learning of the laplacian of the sheaf as an additive separable problem in the restriction maps: 

$$
\min _{L \in \mathcal{L}_F} tr(X^TLX) \equiv \min_{\{\mathcal{F}_{u \triangleleft\ e}, \mathcal{F}_{v \triangleleft e}\}_{e \in \mathcal{E}}} \sum_{e \in \mathcal{E}}|| \mathcal{F}_{u \triangleleft e}X_u - \mathcal{F}_{v \triangleleft e}X_v||_F^2
$$

As a first approach, once we properly defined smooth signals $X$ over the sheaf, is to write each subproblem as 

$$
\min _{\mathcal{F}_{u \triangleleft e}, \mathcal{F}_{v \triangleleft e}} || \mathcal{F}_{u \triangleleft e}X_u - \mathcal{F}_{v \triangleleft e}X_v||_F^2 - \lambda_u \log \det (\mathcal{F}_{u \triangleleft e}^T\mathcal{F}_{u \triangleleft e}) - \lambda_v \log \det (\mathcal{F}_{v \triangleleft e}^T\mathcal{F}_{v \triangleleft e})
$$

the regularization term discourage the presence of any kernel in the maps assumed to be defined from $\mathbb{R}^d$ to $\mathbb{R}^d$: this should prevent the rise of any global section space. 

The optimization scheme in each suproblem is a proximal gradient descent: being 

$$
G(\mathcal{F}_{u \triangleleft e},\mathcal{F}_{v \triangleleft e}) = || \mathcal{F}_{u \triangleleft e}X_u - \mathcal{F}_{v \triangleleft e}X_v||_F^2
$$

$$
\mathcal{R}(\mathcal{F}_{u \triangleleft e}) = - \lambda_u \log \det (\mathcal{F}_{u \triangleleft e}^T\mathcal{F}_{u \triangleleft e})
$$

$$
\mathcal{R}(\mathcal{F}_{v \triangleleft e}) = - \lambda_u \log \det (\mathcal{F}_{v \triangleleft e}^T\mathcal{F}_{v \triangleleft e})
$$
the scheme is:

$$
\mathcal{F}_{u \triangleleft e}^{k+1} = \Pi_{\mathcal{R}(\mathcal{F}_{u \triangleleft e})}[\mathcal{F}_{u \triangleleft e}^{k} - \rho \nabla_{\mathcal{F}_{u \triangleleft e}} G(\mathcal{F}_{u \triangleleft e},\mathcal{F}_{v \triangleleft e})]
$$

$$
\mathcal{F}_{v \triangleleft e}^{k+1} = \Pi_{\mathcal{R}(\mathcal{F}_{v \triangleleft e})}[\mathcal{F}_{v \triangleleft e}^{k} - \rho \nabla_{\mathcal{F}_{v \triangleleft e}} G(\mathcal{F}_{u \triangleleft e},\mathcal{F}_{v \triangleleft e})]
$$


In [2]:
# Optimization routines

def Gradient_Fu(Xu, Xv, Fu, Fv):
    return Fu @ Xu @ Xu.T - Fv @ Xv @ Xu.T

def Gradient_Fv(Xu, Xv, Fu, Fv):
    return Fv @ Xv @ Xv.T - Fu @ Xu @ Xv.T

def ProximalMap(M, Lambda):
    
    def objective(Z, M, lambda_):

        Z = Z.reshape((M.shape[0], M.shape[1]))
        frobenius_term = 0.5 * np.linalg.norm(Z - M, 'fro')**2
        log_det_term = -lambda_ * np.log(np.linalg.det(Z.T @ Z))
        return frobenius_term + log_det_term

    Z0 = np.random.randn(M.shape[0], M.shape[1]).flatten()

    result = minimize(objective, Z0, args=(M, Lambda), method='L-BFGS-B')

    Z_opt = result.x.reshape((M.shape[0], M.shape[1]))

    return Z_opt

def ProxGradDescent(Xu, Xv, d, rho = 5e-3, Lambda = 0.1, MAX_ITER = 50):

    # Initialization

    Fu = np.random.randn(d,d)
    Fv = np.random.randn(d,d)

    for _ in range(MAX_ITER):

        Fu = ProximalMap(Fu - rho * Gradient_Fu(Xu, Xv, Fu, Fv), Lambda)
        Fv = ProximalMap(Fv - rho * Gradient_Fv(Xu, Xv, Fu, Fv), Lambda)

    return Fu, Fv

Let's now define a random graph, a random sheaf over it and signals being smooth over the sheaf. 

In [3]:
# Random graph generation 

def random_ER_graph(
        V:int
        ) -> list:

    edges = []

    for u in range(V):
        for v in range(u+1, V):
            p = np.random.uniform(0,1,1)
            if p < 1.3*np.log(V)/V:
                edges.append((u,v))

    return edges

# Random sheaf generation


def random_sheaf(
        V:int,
        d:int,
        edges:list
        ) -> np.array:

    E = len(edges)

    # Incidency linear maps

    F = {
        e:{
            e[0]:np.random.randn(d,d),
            e[1]:np.random.randn(d,d)
            } 
            for e in edges
        }                                           

    # Coboundary maps

    B = np.zeros((d*E, d*V))                        

    for i in range(len(edges)):

        # Main loop to populate the coboundary map

        edge = edges[i]

        u = edge[0] 
        v = edge[1] 

        B_u = F[edge][u]
        B_v = F[edge][v]

        B[i*d:(i+1)*d, u*d:(u+1)*d] = B_u           
        B[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

    L_f = B.T @ B

    return L_f

# Synthetic data

def synthetic_data(
        N:int, 
        d:int,
        V:int,
        L:np.array
        ) -> np.array:

    # Generate random signals over the stalks of the vertices
    X = np.random.randn(V*d,N)

    # Retrieve the eigendecomposition of the sheaf laplacian
    Lambda, U = np.linalg.eig(L)

    # Tikhonov regularization based approach
    H = 1/(1 + 10*Lambda)

    # Propect into vertices domain <- filter out <- project into spectrum of laplacian
    Y = U @ np.diag(H) @ U.T @ X

    # Add gaussian noise
    Y += np.random.normal(0, 10e-2, size=Y.shape)

    return Y

In [4]:
V = 20
d = 3
N = 1000

In [5]:
G = random_ER_graph(V)
E = len(G)

L = random_sheaf(V, d, G)
Y = synthetic_data(N, d, V, L)

We observe the spectrum of L to assess the structure of the global sections space. 

In [6]:
np.linalg.eigvals(L)

array([3.92616417e+01, 3.33898577e+01, 3.18338221e+01, 3.05379322e+01,
       2.81517088e+01, 2.67379905e+01, 2.64664715e+01, 2.59504052e+01,
       2.42327357e+01, 2.31975931e+01, 2.14905734e+01, 2.04254039e+01,
       1.90765911e+01, 1.73214186e+01, 1.68618074e+01, 1.60276685e+01,
       1.48194325e+01, 1.45186283e+01, 1.40265031e+01, 1.25039132e+01,
       1.16025061e+01, 1.12100044e+01, 1.03680033e+01, 9.96513528e+00,
       9.76044462e+00, 9.34767195e+00, 8.75808936e+00, 7.95260718e+00,
       7.93905952e+00, 7.68352499e+00, 7.30782091e+00, 6.87951598e+00,
       6.76071142e+00, 6.04957408e+00, 2.19637296e-04, 2.78646619e-02,
       1.05706657e-01, 2.48806731e-01, 2.87885395e-01, 6.87921658e-01,
       7.94212520e-01, 8.92764079e-01, 9.69067232e-01, 1.11559829e+00,
       1.07609502e+00, 5.31816103e+00, 4.93542211e+00, 4.88849899e+00,
       4.70287111e+00, 1.80846668e+00, 2.00318148e+00, 2.19624518e+00,
       2.42285228e+00, 4.24578360e+00, 3.90352957e+00, 3.75094038e+00,
      

Let's define a dictionary to keep track of all the maps 

In [7]:
Fs = {
    edge: {
        edge[0]: None,
        edge[1]: None
    }
    for edge in combinations(range(V), 2)
}

In [8]:
for edge in tqdm(combinations(range(V), 2)):
    u = edge[0]
    v = edge[1]

    Xu = Y[u*d:(u+1)*d,:]
    Xv = Y[v*d:(v+1)*d,:]

    Fu, Fv = ProxGradDescent(Xu, Xv, d)

    Fs[edge][u] = Fu
    Fs[edge][v] = Fv

11it [00:03,  2.87it/s]C:\Users\Leonardo\AppData\Local\Temp\ipykernel_12800\1918845247.py:15: RuntimeWarning: invalid value encountered in log
  log_det_term = -lambda_ * np.log(np.linalg.det(Z.T @ Z))
C:\Users\Leonardo\AppData\Local\Temp\ipykernel_12800\1918845247.py:15: RuntimeWarning: divide by zero encountered in log
  log_det_term = -lambda_ * np.log(np.linalg.det(Z.T @ Z))
14it [00:05,  2.25it/s]c:\Users\Leonardo\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\linalg\linalg.py:2139: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
29it [00:11,  2.92it/s]c:\Users\Leonardo\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
190it [01:45,  1.80it/s]


In [9]:
energies = {
    edge: 0 for edge in combinations(range(V),2)
}

In [10]:
for edge in tqdm(combinations(range(V), 2)):
    u = edge[0]
    v = edge[1]

    Xu = Y[u*d:(u+1)*d,:]
    Xv = Y[v*d:(v+1)*d,:]

    Fu = Fs[edge][u]
    Fv = Fs[edge][v]

    energies[edge] = np.linalg.norm(Fu @ Xu - Fv @ Xv)

190it [00:00, 38075.38it/s]


In [11]:
retrieved = sorted(energies.items(), key=lambda x:x[1])[:E]

In [12]:
len(set(list(map(lambda x: x[0], retrieved))).intersection(set(G))) / E

0.24242424242424243

Very poor in recovering the underlying topology!

In [13]:
B_hat = np.zeros((d*E, d*V))

for i in range(E):
    edge = retrieved[i][0]

    u = edge[0] 
    v = edge[1] 

    B_u = Fs[edge][u]
    B_v = Fs[edge][v]

    B_hat[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B_hat[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

L_f_hat = B_hat.T @ B_hat

In [14]:
np.linalg.eigvals(L)

array([3.92616417e+01, 3.33898577e+01, 3.18338221e+01, 3.05379322e+01,
       2.81517088e+01, 2.67379905e+01, 2.64664715e+01, 2.59504052e+01,
       2.42327357e+01, 2.31975931e+01, 2.14905734e+01, 2.04254039e+01,
       1.90765911e+01, 1.73214186e+01, 1.68618074e+01, 1.60276685e+01,
       1.48194325e+01, 1.45186283e+01, 1.40265031e+01, 1.25039132e+01,
       1.16025061e+01, 1.12100044e+01, 1.03680033e+01, 9.96513528e+00,
       9.76044462e+00, 9.34767195e+00, 8.75808936e+00, 7.95260718e+00,
       7.93905952e+00, 7.68352499e+00, 7.30782091e+00, 6.87951598e+00,
       6.76071142e+00, 6.04957408e+00, 2.19637296e-04, 2.78646619e-02,
       1.05706657e-01, 2.48806731e-01, 2.87885395e-01, 6.87921658e-01,
       7.94212520e-01, 8.92764079e-01, 9.69067232e-01, 1.11559829e+00,
       1.07609502e+00, 5.31816103e+00, 4.93542211e+00, 4.88849899e+00,
       4.70287111e+00, 1.80846668e+00, 2.00318148e+00, 2.19624518e+00,
       2.42285228e+00, 4.24578360e+00, 3.90352957e+00, 3.75094038e+00,
      

In [15]:
np.linalg.eigvals(L_f_hat)

array([4.32247061e+01, 3.49989610e+01, 3.02179087e+01, 2.66535298e+01,
       2.55071535e+01, 2.44101167e+01, 2.32508554e+01, 2.24463672e+01,
       2.14863870e+01, 2.08641071e+01, 1.84726422e+01, 1.78864479e+01,
       1.71770500e+01, 1.69150657e+01, 1.60563578e+01, 1.55419728e+01,
       1.45344772e+01, 1.40233346e+01, 1.36864171e+01, 1.31457200e+01,
       1.21049798e+01, 1.12116366e+01, 9.82840099e+00, 9.23300163e+00,
       8.85542558e+00, 8.60597936e+00, 7.66598299e+00, 7.05144387e+00,
       6.61779335e+00, 6.39363482e+00, 1.59725811e-02, 7.37516007e-02,
       5.35685573e-02, 2.33481630e-01, 4.91588057e-01, 3.84891671e-01,
       3.79491870e-01, 7.45987948e-01, 1.26240181e+00, 1.39143118e+00,
       1.67031017e+00, 5.47455993e+00, 2.16358244e+00, 2.49868467e+00,
       5.08324712e+00, 3.25908763e+00, 4.53145908e+00, 3.52046831e+00,
       4.32493160e+00, 4.29678057e+00, 4.07431956e+00, 3.91778516e+00,
       3.81138049e+00, 3.76672451e+00, 0.00000000e+00, 0.00000000e+00,
      

As expected, controlling the spectrum of the restriction maps doesn't imply control over the spectrum of the sheaf laplacian! 